In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv")
print(train.info())

In [ ]:
train_demo = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv")
print(train_demo.info())

In [ ]:
test = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv")
print(test.info())

In [ ]:
test_demo = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv")
print(test_demo.info())

In [ ]:
import os, json, joblib, numpy as np, pandas as pd
from scipy.spatial.transform import Rotation 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from pathlib import Path
import warnings 
warnings.filterwarnings("ignore")



def remove_gravity_from_acc(acc_data, rot_data):

    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    else:
        acc_values = acc_data

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    
    gravity_world = np.array([0, 0, 9.81])

    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :] 
            continue

        try:
            rotation = Rotation.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
             
    return linear_accel

In [ ]:
le = LabelEncoder()
train['gesture_int'] = le.fit_transform(train['gesture']).astype(np.int32)
gesture_classes = le.classes_
train['sequence_id_encode'] = le.fit_transform(train['sequence_id']).astype(np.int32)
sequence_id_classes = le.classes_


In [ ]:
print("  Calculating base engineered IMU features (magnitude, angle)...")
train['acc_mag'] = np.sqrt(train['acc_x']**2 + train['acc_y']**2 + train['acc_z']**2)
train['rot_angle'] = 2 * np.arccos(train['rot_w'].clip(-1, 1))

print("  Calculating engineered IMU derivatives (jerk, angular velocity) for original acc_mag...")
train['acc_mag_jerk'] = train.groupby('sequence_id')['acc_mag'].diff().fillna(0)
train['rot_angle_vel'] = train.groupby('sequence_id')['rot_angle'].diff().fillna(0)

print("  Removing gravity and calculating linear acceleration features...")

linear_accel_list = []
for _, group in train.groupby('sequence_id'):
    acc_data_group = group[['acc_x', 'acc_y', 'acc_z']]
    rot_data_group = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
    linear_accel_group = remove_gravity_from_acc(acc_data_group, rot_data_group)
    linear_accel_list.append(pd.DataFrame(linear_accel_group, columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group.index))

train_linear_accel = pd.concat(linear_accel_list)
train = pd.concat([train, train_linear_accel], axis=1)

train['linear_acc_mag'] = np.sqrt(train['linear_acc_x']**2 + train['linear_acc_y']**2 + train['linear_acc_z']**2)
train['linear_acc_mag_jerk'] = train.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0)

meta_cols = { ... }

imu_cols_base = ['linear_acc_x', 'linear_acc_y', 'linear_acc_z']
imu_cols_base.extend([c for c in train.columns if c.startswith('rot_') and c not in ['rot_angle', 'rot_angle_vel']])

imu_engineered_features = [
    'acc_mag', 'rot_angle',
    'acc_mag_jerk', 'rot_angle_vel',
    'linear_acc_mag', 'linear_acc_mag_jerk'
]
imu_cols = imu_cols_base + imu_engineered_features
imu_cols = list(dict.fromkeys(imu_cols))

In [ ]:
imu_cols.extend(["gesture_int", "sequence_id"])
train_imu = train[imu_cols]
train_imu.info()

In [ ]:
train_imu.groupby('sequence_id')

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

# Group by sequence
grouped = train_imu.groupby("sequence_id")

# Store sequences and labels
X_list = []
y_list = []
seq_lens = []

for seq_id, group in grouped:
    group = group.dropna(subset=imu_cols)  # drop rows with NaNs in IMU features
    if group.empty:
        continue

    features = group[imu_cols].values.astype(np.float32)
    label = group['gesture_int'].iloc[0]

    X_tensor = torch.tensor(features, dtype=torch.float32)
    X_list.append(X_tensor)
    y_list.append(label)
    seq_lens.append(len(X_tensor))

# Compute pad length = 95th percentile
pad_len = int(np.percentile(seq_lens, 95))
print(f"📏 Padding to 95th percentile sequence length: {pad_len}")

# Pad sequences (post-padding)
X_padded = pad_sequence(X_list, batch_first=True)  # (N, max_seq_len, D)

# Truncate or extend to pad_len
if X_padded.size(1) > pad_len:
    X_padded = X_padded[:, :pad_len, :]
elif X_padded.size(1) < pad_len:
    pad_amt = pad_len - X_padded.size(1)
    zero_pad = torch.zeros((X_padded.size(0), pad_amt, X_padded.size(2)))
    X_padded = torch.cat([X_padded, zero_pad], dim=1)

# Label tensor
y_tensor = torch.tensor(y_list, dtype=torch.long)

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X_padded, y_tensor, test_size=0.2, random_state=42, stratify=y_tensor
)

print(f"✅ IMU-only train tensor: {X_train.shape} | val tensor: {X_val.shape}")